In [ ]:
# default_exp quaternions
# default_cls_lvl 3

In [ ]:
#hide
%load_ext line_profiler
%matplotlib notebook

# Tensor Quaternion Module
> Pytorch Models for Sequential Data

In [ ]:
#export
from seqdata.core import *
from seqdata.model import *
from seqdata.learner import *
from fastai2.basics import *
from fastai2.callback.progress import *
from fastai2.callback.schedule import *

## Quaternion Type

In [ ]:
#export
class TensorQuaternionInclination(TensorSequences): pass
class TensorQuaternionAngle(TensorSequences): pass

In [ ]:
f_paths = '/mnt/data/Systemidentification/Orientation_Estimation/'
hdf_files = get_hdf_files(f_paths,recurse=False)
tfm_src = CreateDict([DfHDFCreateWindows(win_sz=1000,stp_sz=100,clm='acc_x')])
u = ['acc_x','acc_y','acc_z','gyr_x','gyr_y','gyr_z']
# u = ['acc_x','acc_y','acc_z','gyr_x','gyr_y','gyr_z','mag_x','mag_y','mag_z']
y =['opt_a','opt_b','opt_c','opt_d']
dls = DataBlock(blocks=(SequenceBlock.from_hdf(u,TensorSequencesInput),
                        SequenceBlock.from_hdf(y,TensorQuaternionInclination)),
                get_items=tfm_src,
                splitter=ApplyToDict(FuncSplitter(lambda o: 'experiment2' in str(o)))
               ).dataloaders(hdf_files,shufflish=True,bs=128)

## Basic Operations

In [ ]:
tq1 = tensor([
    [1,0,0,0],
    [0.5,0.5,0.5,0.5],
    ])
tq2 = tensor([
    [0.5,0.5,0.5,0.5],
    [0.5,0.5,0.5,0.5],
    ])
tq1.shape

torch.Size([2, 4])

In [ ]:
#export
_pi = torch.Tensor([3.14159265358979323846])
def rad2deg(t):
    return 180. * t / _pi.to(t.device).type(t.dtype)

In [ ]:
test_eq(float(rad2deg(_pi)),180)

In [ ]:
#export
@torch.jit.script
def multiplyQuat(q1, q2):
    """quat1*quat2"""
    o1 = q1[..., 0] * q2[..., 0] - q1[..., 1] * q2[..., 1] - q1[..., 2] * q2[..., 2] - q1[..., 3] * q2[..., 3]
    o2 = q1[..., 0] * q2[..., 1] + q1[..., 1] * q2[..., 0] + q1[..., 2] * q2[..., 3] - q1[..., 3] * q2[..., 2]
    o3 = q1[..., 0] * q2[..., 2] - q1[..., 1] * q2[..., 3] + q1[..., 2] * q2[..., 0] + q1[..., 3] * q2[..., 1]
    o4 = q1[..., 0] * q2[..., 3] + q1[..., 1] * q2[..., 2] - q1[..., 2] * q2[..., 1] + q1[..., 3] * q2[..., 0]
    return torch.stack([o1,o2,o3,o4],dim=-1)

In [ ]:
q = tq1.repeat(1000,1)

In [ ]:
%%timeit
torch.cumprod(q,dim=-1)

143 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
%%timeit
[q*q for _ in range(1000)]

8.29 ms ± 21.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
test_eq(multiplyQuat(tq1,tq2),tensor([[ 0.5000,  0.5000,  0.5000,  0.5000],
                                    [-0.5000,  0.5000,  0.5000,  0.5000]]))

In [ ]:
#export
def norm_quaternion(q):
    return q / q.norm( p=2,dim=-1)[...,None]

In [ ]:
test_eq(norm_quaternion(tq1*5),tq1)
test_eq(norm_quaternion(tq1/_pi),tq1)
test_eq(norm_quaternion(tq1[None,...]),tq1[None,...])

In [ ]:
#export
_conjugate_quaternion = tensor([1,-1,-1,-1])

def conjQuat(q):
    return q*_conjugate_quaternion.to(q.device).type(q.dtype)

In [ ]:
test_eq(conjQuat(tq1),tensor([[ 1.0000, -0.0000, -0.0000, -0.0000],
                             [ 0.5000, -0.5000, -0.5000, -0.5000]]))

In [ ]:
#export
def diffQuat(q1,q2,norm=True):
    if norm:
        nq1 = norm_quaternion(q1)
        nq2 = norm_quaternion(q2)
    else:
        nq1 = q1
        nq2 = q2
    return multiplyQuat(nq1, conjQuat(nq2))

In [ ]:
test_eq(diffQuat(tq1,tq2),diffQuat(tq1,tq2*5))
test_ne(diffQuat(tq1,tq2),diffQuat(tq1,tq2*5,norm=False))
test_ne(diffQuat(tq1,tq2),diffQuat(tq1[None,...],tq2[None,...]))

In [ ]:
#export
def safe_acos(t):
    '''numericaly stable variant of arcuscosine'''
    eps = 4e-8 #minimum value for acos(1) != 0
    return t.clamp(-1.0 + eps, 1.0 - eps).acos()

In [ ]:
test_ne(safe_acos(tensor(1.))*1e6,0)
test_eq(safe_acos(tensor(-0.)),_pi/2)

In [ ]:
#export
def inclinationAngle(q1,q2):
    
    q = diffQuat(q1,q2)
    return 2*safe_acos((q[..., 3]**2 + q[..., 0]**2).sqrt())

def relativeAngle(q1,q2):
    q = diffQuat(q1,q2)
    return 2*safe_acos((q[..., 0]).abs())

In [ ]:
print('inclination:', rad2deg(inclinationAngle(tq1,tq2)))
print('relative:', rad2deg(relativeAngle(tq1,tq2)))

inclination: tensor([9.0000e+01, 3.9565e-02])
relative: tensor([1.2000e+02, 3.9565e-02])


In [ ]:
#export
def rollAngle(q1,q2):
    q = diffQuat(q1,q2)
    w,x,y,z= q[...,0],q[...,1],q[...,2],q[...,3]
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    return torch.atan2(t0, t1)

def pitchAngle(q1,q2):
    q = diffQuat(q1,q2)
    w,x,y,z= q[...,0],q[...,1],q[...,2],q[...,3]
    t2 = +2.0 * (w * y - z * x)
    t2 = t2.clamp(-1.0, 1.0 )
    return torch.asin(t2)

In [ ]:
print('roll:', rad2deg(rollAngle(tq1,tq2)))
print('pitch:', rad2deg(pitchAngle(tq1,tq2)))

roll: tensor([0., 0.])
pitch: tensor([-90.,   0.])


In [ ]:
#export
_unit_quaternion = tensor([1.,0,0,0])
def inclinationAngleAbs(q):
    q = diffQuat(q,_unit_quaternion[None,:])
    return 2*((q[..., 3]**2 + q[..., 0]**2).sqrt()).acos()

In [ ]:
rad2deg(inclinationAngleAbs(tq1))

tensor([ 0., 90.])

In [ ]:
#export
def rand_quat():
    q = torch.rand((4))*2-1
    q /= q.norm()
    return q

In [ ]:
#export
def rot_vec(v,q):
    v = F.pad(v, (1,0), "constant", 0)
    return multiplyQuat(conjQuat(q),multiplyQuat(v,q))[...,1:]
#     return multiplyQuat(q,multiplyQuat(v,conjQuat(q)))[...,1:]

In [ ]:
g = tensor([[9.81,0,0]]*5)

In [ ]:
r_quat = rand_quat()
rot_vec(g,r_quat)

tensor([[-2.8532,  0.4130,  9.3768],
        [-2.8532,  0.4130,  9.3768],
        [-2.8532,  0.4130,  9.3768],
        [-2.8532,  0.4130,  9.3768],
        [-2.8532,  0.4130,  9.3768]])

## Loss Functions

In [ ]:
#export
def inclination_loss(q1,q2):
    q = diffQuat(q1,q2)
    q_abs = (q[..., 3]**2 + q[..., 0]**2).sqrt()-1
    return (q_abs**2).mean().sqrt()

In [ ]:
inclination_loss(tq1,tq2)

tensor(0.2071)

In [ ]:
#export
def inclination_loss_abs(q1,q2):
    q = diffQuat(q1,q2)
    q_abs = (q[..., 3]**2 + q[..., 0]**2).sqrt()-1
    return q_abs.abs().mean()

In [ ]:
inclination_loss_abs(tq1,tq2)

tensor(0.1464)

In [ ]:
#export
def inclination_loss_squared(q1,q2):
    q = diffQuat(q1,q2)
    q_abs = (q[..., 3]**2 + q[..., 0]**2).sqrt()-1
    return (q_abs**2).mean()

In [ ]:
# %%timeit
inclination_loss_squared(tq1,tq2)

tensor(0.0429)

In [ ]:
#export
def inclination_loss_smooth(q1,q2):
    q = diffQuat(q1,q2)
    q_abs = (q[..., 3]**2 + q[..., 0]**2).sqrt()-1
    return F.smooth_l1_loss(q_abs,torch.zeros_like(q_abs))

In [ ]:
# %%timeit
inclination_loss_smooth(tq1,tq2)

tensor(0.0214)

In [ ]:
#export
def abs_inclination(q1,q2):
    inclination = inclinationAngle(q1,q2)
    return  inclination.abs().mean()

In [ ]:
abs_inclination(tq1,tq2)

tensor(0.7857)

In [ ]:
#export
def ms_inclination(q1,q2):
    inclination = inclinationAngle(q1,q2)
    return  (inclination**2).mean()

In [ ]:
ms_inclination(tq1,tq2)

tensor(1.2337)

In [ ]:
#export
def rms_inclination(q1,q2):
    inclination = inclinationAngle(q1,q2)
    return (inclination**2).mean().sqrt()

In [ ]:
rms_inclination(tq1,tq2)

tensor(1.1107)

In [ ]:
#export
def smooth_inclination(q1,q2):
    inclination = inclinationAngle(q1,q2)
    return F.smooth_l1_loss(inclination,torch.zeros_like(inclination))

In [ ]:
smooth_inclination(tq1,tq2)

tensor(0.5354)

In [ ]:
#export
def rms_inclination_deg(q1,q2):
    inclination = inclinationAngle(q1,q2)
    return  rad2deg((inclination**2).mean().sqrt())

In [ ]:
rms_inclination_deg(tq1,tq2)

tensor([63.6396])

In [ ]:
#export
def rms_pitch_deg(q1,q2):
    inclination = pitchAngle(q1,q2)
    return  rad2deg((inclination**2).mean().sqrt())

In [ ]:
rms_pitch_deg(tq1,tq2)

tensor([63.6396])

In [ ]:
#export
def rms_roll_deg(q1,q2):
    inclination = rollAngle(q1,q2)
    return  rad2deg((inclination**2).mean().sqrt())

In [ ]:
rms_roll_deg(tq1,tq2)

tensor([0.])

In [ ]:
#export
def mean_inclination_deg(q1,q2):
    inclination = inclinationAngle(q1,q2)
    return  rad2deg(inclination.mean())

In [ ]:
mean_inclination_deg(tq1,tq2)

tensor([45.0198])

In [ ]:
#export
def angle_loss(q1,q2):
    q = diffQuat(q1,q2)
    return (q[..., 0]-1).abs().mean()

In [ ]:
#export
def angle_loss_opt(q1,q2):
    q1 = norm_quaternion(q1)
    q2 = norm_quaternion(q2)
    
    q2 = conjQuat(q2)
    q = q1[..., 0] * q2[..., 0] - q1[..., 1] * q2[..., 1] - q1[..., 2] * q2[..., 2] - q1[..., 3] * q2[..., 3]
    return (q-1).abs().mean()

In [ ]:
#export
def ms_rel_angle(q1,q2):
    rel_angle = relativeAngle(q1,q2)
    return  (rel_angle**2).mean()

In [ ]:
ms_rel_angle(tq1,tq2)

tensor(2.1932)

In [ ]:
#export
def rms_rel_angle_deg(q1,q2):
    rel_angle = relativeAngle(q1,q2)
    return  rad2deg((rel_angle**2).mean().sqrt())

In [ ]:
rms_rel_angle_deg(tq1,tq2)

tensor([84.8528])

In [ ]:
#export
def mean_rel_angle_deg(q1,q2):
    rel_angle = relativeAngle(q1,q2)
    return  rad2deg(rel_angle.mean())

In [ ]:
mean_rel_angle_deg(tq1,tq2)

tensor([60.0198])

In [ ]:
#export
def deg_rmse(inp, targ):
    return rad2deg(fun_rmse(inp, targ))

## Callbacks

In order to assure that the output of the model are close to unit quaternions the distance will be added to the loss

In [ ]:
#export
from fastai2.callback.hook import *
@delegates()
class QuaternionRegularizer(HookCallback):
    "Callback that adds AR and TAR to the loss, calculated by output of provided layer"
    run_before=TrainEvalCallback
    def __init__(self,reg_unit=0.0,detach=False, **kwargs):
        super().__init__(detach=detach,**kwargs)
        store_attr(self,'reg_unit')
        
    def hook(self, m, i, o): 
        if type(o) is torch.Tensor:
            self.out = o
        else:
            self.out = o[0]
    
    def after_loss(self):
        if not self.training: return
        h = self.out.float()
        
        if self.reg_unit != 0.:  
            l_a = float(self.reg_unit) * ((1-h.norm(dim=-1))**2).mean()
#             import pdb; pdb.set_trace()
            self.learn.loss = self.learn.loss+l_a 

In [ ]:
#export
def augmentation_groups(u_groups):
    '''returns the rotation list corresponding to the input groups'''
    u_groups = np.cumsum([0] + u_groups)
    return [[u_groups[i],u_groups[i+1]-1] for i in range(len(u_groups)-1)]

In [ ]:
u_raw_groups = [3,3]
test_eq(augmentation_groups(u_raw_groups),[[0,2],[3,5]])

In [ ]:
#export
class QuaternionAugmentation(Transform):
    "A transform that before_call its state at each `__call__`"
    split_idx = 0
    def __init__(self,inp_groups, **kwargs):
        super().__init__(**kwargs)
        self.inp_groups = inp_groups
        self.r_quat = None
        for g in inp_groups: 
            l = g[1]-g[0]+1
            if l != 4 and l != 3: raise AttributeError

    def __call__(self, b, split_idx=None, **kwargs):
        #import pdb; pdb.set_trace()
        self.r_quat = rand_quat()
        return super().__call__(b, split_idx=split_idx, **kwargs) 

    def encodes(self, x:(TensorSequences)):
        #import pdb; pdb.set_trace()
        for g in self.inp_groups:
            tmp = x[...,g[0]:g[1]+1]
            if tmp.shape[1] == 3:
                x[...,g[0]:g[1]+1] = rot_vec(tmp,self.r_quat)
            else:
                x[...,g[0]:g[1]+1] = multiplyQuat(tmp,self.r_quat)
        return x
    
    def encodes(self, x:(TensorQuaternionInclination,TensorQuaternionAngle)):
        return multiplyQuat(x,self.r_quat)

In [ ]:
n_skip = 2**8

inp,out = get_inp_out_size(dls)
# model = SimpleGRU(inp,out,num_layers=1,hidden_size=100)
model = TCN(inp,out,hl_depth=8,hl_width=10)

skip = partial(SkipNLoss,n_skip=n_skip)
metrics=rms_inclination_deg
cbs = [QuaternionRegularizer(reg_unit=1,modules=[model])]

lrn = Learner(dls,model,loss_func=ms_inclination,opt_func=ranger,metrics=metrics)

In [ ]:
lrn.fit_one_cycle(1,lr_max=3e-3)

## Inclination Datablock

In [ ]:
#export
class TensorInclination(TensorSequences): pass

In [ ]:
#export
import h5py
class HDF2Inclination(HDF2Sequence):
        
    def _hdf_extract_sequence(self,hdf_path,dataset = None, l_slc = None, r_slc= None,down_s=None):
        with h5py.File(hdf_path,'r') as f:
            ds = f if dataset is None else f[dataset]
            l_array = [ds[n][l_slc:r_slc] for n in self.clm_names]
            seq = np.vstack(l_array).T
            seq = array(inclinationAngleAbs(tensor(seq))[:,None])
            return seq

In [ ]:
#export
class InclinationBlock(TransformBlock):
    def __init__(self, seq_extract,padding=False):
        return super().__init__(type_tfms=[seq_extract],
                                batch_tfms=[Normalize(axes=[0,1])],
                                dls_kwargs={} if not padding else {'before_batch': pad_sequence})

    @classmethod
    @delegates(HDF2Inclination, keep=True)
    def from_hdf(cls, clm_names, seq_cls=TensorInclination,padding=False, **kwargs):
        return cls(HDF2Inclination(clm_names,to_cls=seq_cls,**kwargs), padding)


In [ ]:
# f_paths = '/mnt/Data/Systemidentification/Orientation_Estimation/'
# hdf_files = get_hdf_files(f_paths)
# tfm_src = CreateDict([DfHDFCreateWindows(win_sz=1000,stp_sz=100,clm='acc_x')])
# u = ['acc_x','acc_y','acc_z','gyr_x','gyr_y','gyr_z']
# # u = ['acc_x','acc_y','acc_z','gyr_x','gyr_y','gyr_z','mag_x','mag_y','mag_z']
# y =['opt_a','opt_b','opt_c','opt_d']
# dls = DataBlock(blocks=(SequenceBlock.from_hdf(u),
#                         InclinationBlock.from_hdf(y)),
#                 get_items=tfm_src,
#                 splitter=ApplyToDict(FuncSplitter(lambda o: 'experiment2' in str(o)))
#                ).dataloaders(hdf_files,shufflish=True,bs=128)

## Show Results

In [ ]:
#export
def plot_scalar_inclination(axs,in_sig,targ_sig,out_sig=None, **kwargs):
#     import pdb; pdb.set_trace()
    first_targ = targ_sig[0].repeat(targ_sig.shape[0],1)
    axs[0].plot(rad2deg(targ_sig))
    axs[0].label_outer()
    axs[0].set_ylabel('inclination[°]')
    
    if out_sig is not None:
        axs[0].plot(rad2deg(out_sig))
        axs[0].legend(['y','ŷ'])
        axs[1].plot(rad2deg(targ_sig-out_sig))
        axs[1].label_outer()
        axs[1].set_ylabel('error[°]')
        
    axs[-1].plot(in_sig)

In [ ]:
#export
def plot_quaternion_inclination(axs,in_sig,targ_sig,out_sig=None, **kwargs):
#     import pdb; pdb.set_trace()
    axs[0].plot(rad2deg(inclinationAngleAbs(targ_sig)))
    axs[0].label_outer()
    axs[0].legend(['y'])
    axs[0].set_ylabel('inclination[°]')
    
    if out_sig is not None:
        axs[0].plot(rad2deg(inclinationAngleAbs(out_sig)))
        axs[0].legend(['y','ŷ'])
        axs[1].plot(rad2deg(inclinationAngle(out_sig,targ_sig)))
        axs[1].label_outer()
        axs[1].set_ylabel('error[°]')
        if 'ref' in kwargs:
#             axs[0].plot(rad2deg(inclinationAngleAbs(kwargs['ref'])))
#             axs[0].legend(['y','ŷ','y_ref'])
            axs[1].plot(rad2deg(inclinationAngle(targ_sig,kwargs['ref'])))
            axs[1].legend(['ŷ','y_ref'])
        
    axs[-1].plot(in_sig)

In [ ]:
#export
def plot_quaternion_rel_angle(axs,in_sig,targ_sig,out_sig=None, **kwargs):
#     import pdb; pdb.set_trace()
    first_targ = targ_sig[0].repeat(targ_sig.shape[0],1)
    axs[0].plot(rad2deg(relativeAngle(first_targ,targ_sig)))
    axs[0].label_outer()
    axs[0].legend(['y'])
    axs[0].set_ylabel('angle[°]')
    
    if out_sig is not None:
        axs[0].plot(rad2deg(relativeAngle(first_targ,out_sig)))
        axs[0].legend(['y','ŷ'])
        axs[1].plot(rad2deg(relativeAngle(out_sig,targ_sig)))
        axs[1].label_outer()
        axs[1].set_ylabel('error[°]')
        
    axs[-1].plot(in_sig)

In [ ]:
dls.show_batch(max_n=3,ds_idx=0)

In [ ]:
#export
@typedispatch
def show_results(x:TensorSequences, y:TensorInclination, samples, outs, ctxs=None, max_n=2, **kwargs):
    n_samples = min(len(samples), max_n)
    n_targ = 2
    if n_samples > 3:
        #if there are more then 3 samples to plot then put them in a single figure
        plot_seqs_single_figure(n_samples,n_targ,samples,plot_scalar_inclination,outs, **kwargs)
    else:
        #if there are less then 3 samples to plot then put each in its own figure
        plot_seqs_multi_figures(n_samples,n_targ,samples,plot_scalar_inclination,outs, **kwargs)
    return ctxs

In [ ]:
#export
@typedispatch
def show_batch(x:TensorSequences, y:TensorInclination, samples, ctxs=None, max_n=6, **kwargs):
    n_samples = min(len(samples), max_n)
    n_targ = 1
    if n_samples > 3:
        #if there are more then 3 samples to plot then put them in a single figure
        plot_seqs_single_figure(n_samples,n_targ,samples,plot_scalar_inclination, **kwargs)
    else:
        #if there are less then 3 samples to plot then put each in its own figure
        plot_seqs_multi_figures(n_samples,n_targ,samples,plot_scalar_inclination, **kwargs)
    return ctxs

In [ ]:
#export
@typedispatch
def show_results(x:TensorSequences, y:TensorQuaternionInclination, samples, outs, ctxs=None, max_n=2, **kwargs):
    if 'quat' in kwargs: return show_results(x,TensorSequencesOutput(y), samples,outs, ctxs, max_n , **kwargs)
    n_samples = min(len(samples), max_n)
    n_targ = 2
    if n_samples > 3:
        #if there are more then 3 samples to plot then put them in a single figure
        plot_seqs_single_figure(n_samples,n_targ,samples,plot_quaternion_inclination,outs,**kwargs)
    else:
        #if there are less then 3 samples to plot then put each in its own figure
        plot_seqs_multi_figures(n_samples,n_targ,samples,plot_quaternion_inclination,outs,**kwargs)
    return ctxs

In [ ]:
#export
@typedispatch
def show_batch(x:TensorSequences, y:TensorQuaternionInclination, samples, ctxs=None, max_n=6, **kwargs):
    n_samples = min(len(samples), max_n)
    n_targ = 1
    if n_samples > 3:
        #if there are more then 3 samples to plot then put them in a single figure
        plot_seqs_single_figure(n_samples,n_targ,samples,plot_quaternion_inclination)
    else:
        #if there are less then 3 samples to plot then put each in its own figure
        plot_seqs_multi_figures(n_samples,n_targ,samples,plot_quaternion_inclination)
    return ctxs

In [ ]:
#export
@typedispatch
def show_results(x:TensorSequences, y:TensorQuaternionAngle, samples, outs, ctxs=None, max_n=2, **kwargs):
    n_samples = min(len(samples), max_n)
    n_targ = 2
    if n_samples > 3:
        #if there are more then 3 samples to plot then put them in a single figure
        plot_seqs_single_figure(n_samples,n_targ,samples,plot_quaternion_rel_angle,outs, **kwargs)
    else:
        #if there are less then 3 samples to plot then put each in its own figure
        plot_seqs_multi_figures(n_samples,n_targ,samples,plot_quaternion_rel_angle,outs, **kwargs)
    return ctxs

In [ ]:
#export
@typedispatch
def show_batch(x:TensorSequences, y:TensorQuaternionAngle, samples, ctxs=None, max_n=6, **kwargs):
    n_samples = min(len(samples), max_n)
    n_targ = 1
    if n_samples > 3:
        #if there are more then 3 samples to plot then put them in a single figure
        plot_seqs_single_figure(n_samples,n_targ,samples,plot_quaternion_rel_angle, **kwargs)
    else:
        #if there are less then 3 samples to plot then put each in its own figure
        plot_seqs_multi_figures(n_samples,n_targ,samples,plot_quaternion_rel_angle, **kwargs)
    return ctxs

In [ ]:
lrn.show_results(max_n=3,ds_idx=0,shuffle=True,quat=True)

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_model.ipynb.
Converted 02_learner.ipynb.
Converted 03_tbptt_dl.ipynb.
Converted 11_dualrnn.ipynb.
Converted 12_TensorQuaternions.ipynb.
Converted 13_HPOpt.ipynb.
Converted index.ipynb.
